In [ ]:
# Original neural-style documentation: https://github.com/jcjohnson/neural-style

import os, math
from PIL import Image
from IPython.display import display, clear_output

_DEBUG_MODE_ = True

script_style = ['/usr/local/bin/th', 'neural_style_dir_rng_fix.lua']
script_lincolor = ['python3', 'Neural-Tools/linear-color-transfer.py']
script_lum = ['python3', 'Neural-Tools/lum-transfer.py']

model_vgg = 'models/VGG_ILSVRC_19_layers.caffemodel'
model_vggnorm = 'models/vgg_normalised.caffemodel'
model_nin = 'models/nin_imagenet_conv.caffemodel'
model_nyud = 'models/nyud-fcn32s-color-heavy.caffemodel'
model_channelpruning = 'models/channel_pruning.caffemodel'

proto_vgg = 'models/VGG_ILSVRC_19_layers_deploy.prototxt'
proto_vggnorm = 'models/VGG_ILSVRC_19_layers_deploy_fullconv.prototxt'
proto_nin = 'models/train_val.prototxt'
proto_nyud = 'models/nyud-fcn32s-color-heavy-trainval.prototxt'
proto_channelpruning = 'models/channel_pruning.prototxt'

slayer_vgg = 'relu1_1,relu2_1,relu3_1,relu4_1,relu5_1'
slayer_vggnorm = slayer_vgg
slayer_nin = 'relu0,relu1,relu2,relu3,relu5,relu6,relu7,relu8'
slayer_nyud = slayer_vgg
slayer_channelpruning = slayer_vgg

clayer_vgg = 'relu4_2'
clayer_vggnorm = clayer_vgg
clayer_nin = 'relu0,relu1,relu2,relu3,relu5,relu6,relu7,relu8'
clayer_nyud = clayer_vgg
clayer_channelpruning = clayer_vgg


def show_image(image_path):
    with Image.open(image_path) as im:
        display(im)

def style_transfer(style_image,
                   content_image,
                   image_size=None, #defaults to 512
                   style_blend_weights=None, #defaults to equal weights
                   gpu=None, #defaults to 0, use -1 for CPU only (why would you?)
                   content_weight=None, #defaults to '5e0'
                   style_weight=None, #defaults to '1e2'
                   tv_weight=None, #defaults to 1e-3
                   num_iterations=None, #defaults to 1000
                   init=None, #defaults to 'random', pass 'image' to use init_image
                   init_image=None,
                   optimizer=None, #defaults to 'lbfgs', 'adam' is lower mem but worse
                   learning_rate=None, #defaults to '1e1', for ADAM only
                   normalize_gradients=None, #defaults to unset, good for ADAM
                   output_image='out.png', #defaults to 'out.png'
                   print_iter=None, #defaults to 50, 0 to disable
                   save_iter=None, #defaults to 100, 0 to disable
                   content_layers=None, #defaults to 'relu4_2'
                   style_layers=None, #defaults to 'relu1_1,relu2_1,relu3_1,relu4_1,relu5_1'
                   style_scale=None, #defaults to 1.0
                   original_colors=None, #defaults to 0, 1 for original colors
                   proto_file=None, #defaults to 'models/VGG_ILSVRC_19_layers_deploy.prototxt'
                   model_file=None, #defaults to 'models/VGG_ILSVRC_19_layers.caffemodel'
                   pooling=None, #default is 'max', 'avg' may or may not get better results
                   backend=None, #default is 'nn', 'cudnn' and 'clnn' are other options
                   cudnn_autotune=None, #default is unset, more mem but faster backend
                   multigpu_strategy=None, #default is '', see original documentation
                   lbfgs_num_correction=None, #default is 0, higher gets better results I think
                   seed=None): #default is -1, for random I believe
    
    arg_names = ['style_image','content_image','image_size','style_blend_weights','gpu',
                 'content_weight','style_weight','tv_weight','num_iterations','init', 'init_image',
                 'optimizer','learning_rate','normalize_gradients','output_image',
                 'print_iter','save_iter','content_layers','style_layers','style_scale',
                 'original_colors','proto_file','model_file','pooling','backend',
                 'cudnn_autotune','multigpu_strategy','lbfgs_num_correction','seed']
    
    command = script_style.copy()
    for arg_name in arg_names:
        arg_val = eval(arg_name)
        if arg_val != None:
            arg_val = str(arg_val)
            command.append('-' + arg_name)
            if arg_name not in ['normalize_gradients', 'cudnn_autotune']:
                command.append(arg_val)
    
    command = ' '.join(command)
    if _DEBUG_MODE_:
        print('\n' + command + '\n')
    !{command}
    
    if os.path.isfile(output_image):
        if _DEBUG_MODE_:
            show_image(output_image)
        print('Transfer complete! Saved to "{}"'.format(output_image)
    else:
        print('Transfer FAILED!')

def ez_cuda(cuda):
    r = dict()
    if cuda:
        r['backend'] = 'cudnn'
        r['cudnn_autotune'] = True
    else:
        r['backend'] = 'nn'
        r['cudnn_autotune'] = None
    return r

def ez_model(model, clayers_override=None, slayers_override=None):
    r = dict()
    if model in ['vgg', 'vggnorm', 'nin', 'nyud', 'channelpruning']:
        r['model_file'] = eval('model_' + model)
        r['proto_file'] = eval('proto_' + model)
        
        if clayers_override != None:
            r['content_layers'] = clayers_override
        else:
            r['content_layers'] = eval('clayer_' + model)

        if slayers_override != None:
            r['style_layers'] = slayers_override
        else:
            r['style_layers'] = eval('slayer_' + model)
        
    else:
        raise Exception('"{}" is not a valid model name.'.format(model))
    return r
    
def ez_style_transfer(style_image,
                      content_image,
                      image_size=None, #defaults to 512
                      style_blend_weights=None, #defaults to equal weights
                      gpu=None, #defaults to 0, use -1 for CPU only (why would you?)
                      content_weight=None, #defaults to '5e0'
                      style_weight=None, #defaults to '1e2'
                      tv_weight=None, #defaults to 1e-3
                      num_iterations=None, #defaults to 1000
                      init=None, #defaults to 'random', pass 'image' to use init_image
                      init_image=None,
                      optimizer=None, #defaults to 'lbfgs', 'adam' is lower mem but worse
                      learning_rate=None, #defaults to '1e1', for ADAM only
                      normalize_gradients=None, #defaults to unset, good for ADAM
                      output_image=None, #defaults to 'out.png'
                      print_iter=None, #defaults to 50, 0 to disable
                      save_iter=None, #defaults to 100, 0 to disable
                      style_scale=None, #defaults to 1.0
                      original_colors=None, #defaults to 0, 1 for original colors
                      pooling=None, #default is 'max', 'avg' may or may not get better results
                      multigpu_strategy=None, #default is '', see original documentation
                      lbfgs_num_correction=None, #default is 0, higher gets better results I think
                      seed=None, #default is -1, for random I believe
                      
                      cuda=True,
                      model='vgg', #Options: 'vgg', 'vggnorm', 'nyud', 'nin', 'channelpruning'
                      clayers_override=None,
                      slayers_override=None): 
    
    m = ez_model(model,
                 clayers_override = clayers_override,
                 slayers_override = slayers_override)
    c = ez_cuda(cuda)
        
    r = style_transfer(style_image=style_image,
                       content_image=content_image,
                       image_size=image_size, 
                       style_blend_weights=style_blend_weights, 
                       gpu=gpu, 
                       content_weight=content_weight, 
                       style_weight=style_weight, 
                       tv_weight=tv_weight, 
                       num_iterations=num_iterations, 
                       init=init,
                       init_image=init_image,
                       optimizer=optimizer, 
                       learning_rate=learning_rate, 
                       normalize_gradients=normalize_gradients, 
                       output_image=output_image, 
                       print_iter=print_iter, 
                       save_iter=save_iter, 
                       content_layers=m['content_layers'], 
                       style_layers=m['style_layers'], 
                       style_scale=style_scale, 
                       original_colors=original_colors, 
                       proto_file=m['proto_file'],
                       model_file=m['model_file'],
                       pooling=pooling,
                       backend=c['backend'],
                       cudnn_autotune=c['cudnn_autotune'],
                       multigpu_strategy=multigpu_strategy,
                       lbfgs_num_correction=lbfgs_num_correction,
                       seed=seed)

def image_max_size(image_in):
    with Image.open(image_in) as im:
        im_size = im.size
    return max(im_size)

def multires_transfer(style_image,
                      content_image,
                      style_blend_weights=None, #defaults to equal weights
                      gpu=None, #defaults to 0, use -1 for CPU only (why would you?)
                      content_weight=None, #defaults to '5e0'
                      style_weight=None, #defaults to '1e2'
                      tv_weight=None, #defaults to 1e-3
                      num_iterations=None,
                      init=None, #defaults to 'random', pass 'image' to use init_image
                      init_image=None,
                      optimizer=None, #defaults to 'lbfgs', 'adam' is lower mem but worse
                      learning_rate=None, #defaults to '1e1', for ADAM only
                      normalize_gradients=None, #defaults to unset, good for ADAM
                      output_image=None, #defaults to 'out.png'
                      print_iter=None, #defaults to 50, 0 to disable
                      content_layers=None, #defaults to 'relu4_2'
                      style_layers=None, #defaults to 'relu1_1,relu2_1,relu3_1,relu4_1,relu5_1'
                      style_scale=None, #defaults to 1.0
                      original_colors=None, #defaults to 0, 1 for original colors
                      proto_file=None, #defaults to 'models/VGG_ILSVRC_19_layers_deploy.prototxt'
                      model_file=None, #defaults to 'models/VGG_ILSVRC_19_layers.caffemodel'
                      pooling=None, #defauls to 'max', 'avg' may or may not get better results
                      multigpu_strategy=None, #default is '', see original documentation
                      lbfgs_num_correction=None, #defauls to 0, higher gets better results I think
                      backend=None,
                      cudnn_autotune=None,
                      
                      start_size=512,
                      final_size=None, #defaults to original content size
                      step_size=250):
    
    if final_size == None:
        final_size = image_max_size(content_image)
    
    steps = math.ceil((final_size - start_size) // step_size)
    
    for step in range(steps):
        if step == steps - 1:
            curr_size = final_size
        else:
            curr_size = (step * step_size) + start_size
        
        if init != None:
            if step != 0:
                init_image = str(step-1) + '.png'
        
        if step == steps-1:
            img_out = output_image
        else:
            img_out = str(step) + '.png'
        
        print('Starting multires transfer step {}/{}...'.format(step + 1, steps))
        
        r = style_transfer(style_image=style_image,
                           content_image=content_image,
                           image_size=curr_size, 
                           style_blend_weights=style_blend_weights, 
                           gpu=gpu, 
                           content_weight=content_weight, 
                           style_weight=style_weight, 
                           tv_weight=tv_weight, 
                           num_iterations=num_iterations, 
                           init=init,
                           init_image=init_image,
                           optimizer=optimizer, 
                           learning_rate=learning_rate, 
                           normalize_gradients=normalize_gradients, 
                           output_image=img_out, 
                           print_iter=print_iter, 
                           save_iter=0, 
                           content_layers=content_layers, 
                           style_layers=style_layers, 
                           style_scale=style_scale, 
                           original_colors=original_colors, 
                           proto_file=proto_file,
                           model_file=model_file,
                           pooling=pooling,
                           backend=backend,
                           cudnn_autotune=cudnn_autotune,
                           multigpu_strategy=multigpu_strategy,
                           lbfgs_num_correction=lbfgs_num_correction,
                           seed=-1)
        
        
        if not _DEBUG_MODE_:
            clear_output()
        else:    
            show_image(img_out)
        
        if os.path.isfile(img_out):
            if step > 0:
                os.remove(init_image)
            
            if step < steps - 1:
                print('Step {}/{} complete! Proceeding...'.format(step+1, steps))
            else:
                print('All {} steps complete!'.format(steps))
                print('Final image saved to "{}"'.format(output_image))
        else:
            print('Step {}/{} FAILED!'.format(step+1, steps))
            if step > 0:
                os.rename(init_image, output_image)
                print('Image from previous step saved to "{}"'.format(output_image))
            return

def ez_multires_transfer(style_image,
                         content_image,
                         style_blend_weights=None, #defaults to equal weights
                         gpu=None, #defaults to 0, use -1 for CPU only (why would you?)
                         content_weight=None, #defaults to '5e0'
                         style_weight=None, #defaults to '1e2'
                         tv_weight=None, #defaults to 1e-3
                         num_iterations=None, #defaults to 1000
                         init=None, #defaults to 'random', pass 'image' to use init_image
                         init_image=None,
                         optimizer=None, #defaults to 'lbfgs', 'adam' is lower mem but worse
                         learning_rate=None, #defaults to '1e1', for ADAM only
                         normalize_gradients=None, #defaults to unset, good for ADAM
                         output_image=None, #defaults to 'out.png'
                         print_iter=None, #defaults to 50, 0 to disable
                         style_scale=None, #defaults to 1.0
                         original_colors=None, #defaults to 0, 1 for original colors
                         pooling=None, #defauls to 'max', 'avg' may or may not get better results
                         multigpu_strategy=None, #default is '', see original documentation
                         lbfgs_num_correction=None, #defauls to 0, higher gets better results I think
                      
                         start_size=512,
                         final_size=None, #defaults to original content size
                         step_size=250,
                         cuda=True,
                         model='vgg',
                         clayers_override=None,
                         slayers_override=None):
    
    m = ez_model(model,
        clayers_override = clayers_override,
        slayers_override = slayers_override)
    c = ez_cuda(cuda)
    
    r = multires_transfer(style_image,
                          content_image,
                          style_blend_weights=style_blend_weights,
                          gpu=gpu,
                          content_weight=content_weight,
                          style_weight=style_weight, 
                          tv_weight=tv_weight,
                          num_iterations=num_iterations,
                          init=init, 
                          init_image=init_image,
                          optimizer=optimizer, 
                          learning_rate=learning_rate, 
                          normalize_gradients=normalize_gradients,
                          output_image=output_image,
                          print_iter=print_iter,
                          content_layers=m['content_layers'],
                          style_layers=m['style_layers'],
                          style_scale=style_scale,
                          original_colors=original_colors,
                          proto_file=m['proto_file'],
                          model_file=m['model_file'],
                          pooling=pooling,
                          multigpu_strategy=multigpu_strategy,
                          lbfgs_num_correction=lbfgs_num_correction,
                          backend=c['backend'],
                          cudnn_autotune=c['cudnn_autotune'],
                          start_size=start_size,
                          final_size=final_size,
                          step_size=step_size)

print('Notebook ready!')

In [ ]:
ez_style_transfer(style_image = 'test_files/style/',
                  content_image = 'test_files/content.jpg',
                  style_weight = '1000',
                  content_weight = '5',
                  tv_weight = 0.00001,
                  num_iterations = 1000,
                  image_size = 512,
                  print_iter = 50,
                  save_iter = 0,
                  model = 'nyud',
                  output_image = 'start.png',
                  normalize_gradients = True,
                  clayers_override = 'relu1_1,relu2_1,relu3_1,relu4_1,relu5_1',
                  slayers_override = 'relu1_1,relu2_1,relu3_1,relu4_1,relu5_1')

print('Step complete!')

In [ ]:
ez_style_transfer(style_image = 'test_files/style/',
                  content_image = 'test_files/content.jpg',
                  style_weight = '100',
                  content_weight = '375',
                  tv_weight = 0.00001,
                  num_iterations = 500,
                  image_size = 800,
                  print_iter = 50,
                  save_iter = 0,
                  model = 'nyud',
                  init = 'image',
                  init_image = 'start.png',
                  output_image = 'start2.png',
                  normalize_gradients = True,
                  clayers_override = 'relu1_1,relu2_1,relu3_1,relu4_1,relu5_1',
                  slayers_override = 'relu1_1,relu2_1,relu3_1,relu4_1,relu5_1')

print('Step complete!')

In [ ]:
ez_style_transfer(style_image = 'test_files/style/',
                  content_image = 'test_files/content.jpg',
                  style_weight = '1000',
                  content_weight = '0',
                  tv_weight = 0,
                  num_iterations = 200,
                  image_size = 1148,
                  print_iter = 50,
                  save_iter = 0,
                  model = 'nyud',
                  init = 'image',
                  init_image = 'start2.png',
                  output_image = 'start3.png',
                  normalize_gradients = True,
                  optimizer = 'adam',
                  learning_rate = 1)

print('Step complete!')

In [ ]:
ez_style_transfer(style_image = 'test_files/style/',
                  content_image = 'test_files/content.jpg',
                  style_weight = '1000',
                  content_weight = '0',
                  tv_weight = 0,
                  num_iterations = 600,
                  image_size = 1800,
                  print_iter = 50,
                  save_iter = 0,
                  model = 'channelpruning',
                  output_image = 'pruned.png',
                  init = 'image',
                  init_image = 'start3.png',
                  normalize_gradients = True,
                  optimizer = 'adam',
                  learning_rate = 1)

print('Step complete!')

In [ ]:
ez_multires_transfer(style_image = 'test_files/style/',
                     content_image = 'test_files/content.jpg',
                     style_weight = '1000',
                     content_weight = '0',
                     tv_weight = 0,
                     num_iterations = 200,
                     output_image = 'result.png',
                     start_size = 2500,
                     step_size = 250,
                     init = 'image',
                     init_image = 'pruned.png',
                     final_size = 4500,
                     print_iter = 50,
                     model = 'nin',
                     normalize_gradients = True,
                     optimizer = 'adam',
                     learning_rate = 1)

print('Step complete!')